# Project for SYSC5001

### Simulation of a manufacturing plant

### Importing Libraries

In [1]:
import pandas as pd
from math import log, pow
import random

### Creating the workstation variables and functions 

In [2]:
class workstation:
    def __init__(self,param):
        self.stat = 0
        self.bufferC1 = 0
        self.bufferC2 = 0
        self.bufferC3 = 0
        self.work_time = 0
        self.start_time = 0
        self.Production = 0
        self.parameters = param
    def start_ws(self, W,clock):
        #get_random time
        b,a,c = self.parameters
        if a > 1:
            time = c*random.weibullvariate(a,b)
        else:
            time = c*random.expovariate(a)
        #change status
        self.stat = 1
        self.start_time = clock
        self.bufferC1 -= 1          #Consume 1 component from the buffer C1
        if W == 'w2':
            self.bufferC2 -= 1      #Consume 1 component from the buffer C2
        elif W == 'w3':
            self.bufferC3 -= 1      #Consume 1 component from the buffer C3
        #schedule next event
        next_event = [time+clock, W,'Finish '+W]
        return next_event
    def end_ws(self, clock):
        #change status
        self.stat = 0
        self.work_time = self.work_time + clock - self.start_time
        self.Production += 1
        
        

### Creating the Inspectors variables and functions

In [3]:
class inspector:
    def __init__(self):
        self.stat = 0
        self.block = 0
        self.work_time = 0
        self.start_time = 0
        self.C1 = 0 #Component waiting to deliver
        self.C2 = 0 #Component waiting to deliver
        self.C3 = 0 #Component waiting to deliver
    def start_ipc(self, C, parameters, clock):
        #get_random time
        b,a,c = parameters
        if a > 1:
            time = c*random.weibullvariate(a,b)
        else:
            time = c*random.expovariate(a)
        #change status
        self.stat = 1
        self.start_time = clock
        #schedule next event
        next_event = [time+clock,'insp'+C[1],'Finish '+C]
        return next_event
    def end_ipc(self, clock):
        self.stat = 0
        self.work_time = self.work_time + clock - self.start_time 
        #change status

## Insert new event to the Event List

In [4]:
def event_list(lista):
    lista = [lista[0],['w1={}'.format(w1.stat),
                  'w1_buffer_C1={}'.format(w1.bufferC1),
                  'w2={}'.format(w2.stat,w2.bufferC1),
                  'w2_bufferC1={}'.format(w2.bufferC1),
                  'w2_bufferC2={}'.format(w2.bufferC2),
                  'w3={}'.format(w3.stat),
                  'w3_bufferC1={}'.format(w3.bufferC1),
                  'w3_bufferC3={}'.format(w3.bufferC3),
                  'Insp1={}'.format(insp1.stat),
                  'Insp1_Blocked={}'.format(insp1.block),
                  'Insp2={}'.format(insp2.stat),
                  'Insp2_Blocked={}'.format(insp2.block),
                 ],queue,lista[1],lista[2],['w1_Prod={}'.format(w1.Production),
                                           'w2_Prod={}'.format(w2.Production),
                                           'w3_Prod={}'.format(w3.Production)]]
    
    Events.loc[-1] = lista  # adding a row
    Events.index = Events.index + 1  # shifting index
    Events.reset_index(inplace=True)
    Events.drop('index',axis=1, inplace=True)

## Insert new event to the Future Event queue

In [5]:
def insert(lista):
    queue.loc[-1] = lista  # adding a row
    queue.index = queue.index + 1  # shifting index
    queue.sort_values(by='Clock',axis=0, inplace=True)
    queue.reset_index(inplace=True)
    queue.drop('index',axis=1, inplace=True)

## Getting and dropping first from list

In [6]:
def next_event():
    nexteve = queue.loc[0]
    queue.drop(0,axis=0,inplace=True)
    queue.reset_index(inplace=True)
    queue.drop('index',axis=1, inplace=True)
    return list(nexteve)

## initialization

In [7]:
count = 0
Clock = 0
event = ['','','']
queue = pd.DataFrame(columns=['Clock', 'Entity', 'Event'])
Events = pd.DataFrame(columns=['Clock', 'System State','FEQ','Entity', 'Event','Production'])
w1 = workstation([1,0.195,1])
w2 = workstation([1,13.4,1])
w3 = workstation([1,9.6,1])
insp1 = inspector()
insp2 = inspector()

## Simulation Loop

In [8]:
while True: 
    #Getting a new event from the list
    if queue.size:             #Check if is not empty
        event = next_event()
        Clock = event[0]           #Updating Clock
    #Executing Change of states triggered by the event
    if event[2] == 'Finish C1':
        insp1.end_ipc(Clock)
        event_list([Clock, event[1], event[2]])
        if w1.bufferC1 <= w2.bufferC1 and w1.bufferC1 <= w3.bufferC1 and w1.bufferC1 < 2:
            w1.bufferC1 += 1
        elif w2.bufferC1 <= w3.bufferC1 and w2.bufferC1 < 2:
            w2.bufferC1 += 1
        elif w3.bufferC1 < 2:
            w3.bufferC1 += 1
        else:
            insp1.block = 1
            insp1.C1 = 1
    if event[2] == 'Finish C2':
        insp2.end_ipc(Clock)
        event_list([Clock, event[1], event[2]])
        if w2.bufferC2 < 2:
            w2.bufferC2 += 1
        else:
            insp2.block = 1
            insp2.C2 = 1
    if event[2] == 'Finish C3':
        insp2.end_ipc(Clock)
        event_list([Clock, event[1], event[2]])
        if w3.bufferC3 < 2:
            w3.bufferC3 += 1
        else:
            insp2.block = 1
            insp2.C3 = 1
    if event[2] == 'Finish w1':
        w1.end_ws(Clock)
        event_list([Clock, event[1], event[2]])
    if event[2] == 'Finish w2':
        w2.end_ws(Clock)
        event_list([Clock, event[1], event[2]])
    if event[2] == 'Finish w3':
        w3.end_ws(Clock)
        event_list([Clock, event[1], event[2]])
    
    #Check for system status and initialyse activities if possible
    #inspector 1                                          #Check if inspector can start new activity
    busy1 = insp1.stat
    blocked1 = insp1.block
    if not busy1 and not blocked1: 
        insert(insp1.start_ipc('C1', [1,11.8,1], Clock))              #Schedule finishing event
        event_list([Clock,'insp1','Starting C1'])
    #inspector 2                                          #Check if inspector can start new activity
    busy2 = insp2.stat
    blocked2 = insp2.block
    if not busy2 and not blocked1:
        rand = random.randint(0,1)
        if rand == 1:
            insert(insp2.start_ipc('C2', [1,17.8,1], Clock))          #Schedule finishing event
            event_list([Clock,'insp2','Starting C2'])
        else:
            insert(insp2.start_ipc('C3', [1,0.0458,1], Clock))          #Schedule finishing event
            event_list([Clock,'insp2','Starting C3'])
    #Workstation 1 -----------------------------------------Check if worksation can start new activity
    busy1 = w1.stat
    C1 = w1.bufferC1
    if not busy1 and (C1>0):
        insert(w1.start_ws('w1', Clock))              #Schedule finishing event
        event_list([Clock,'w1','Starting w1'])
    #Workstation 2 -----------------------------------------Check if worksation can start new activity
    busy2 = w2.stat
    C1 = w2.bufferC1
    C2 = w2.bufferC2
    if not busy2 and (C1>0) and (C2>0):
        insert(w2.start_ws('w2', Clock))              #Schedule finishing event
        event_list([Clock,'w2','Starting w2'])
    #Workstation 3 -----------------------------------------Check if worksation can start new activity
    busy3 = w3.stat
    C1 = w3.bufferC1
    C3 = w3.bufferC3
    if not busy3 and (C1>0) and (C3>0):
        insert(w3.start_ws('w3', Clock))              #Schedule finishing event
        event_list([Clock,'w3','Starting w3'])
        
    #Checking if inspectors can be unblocked
    if insp1.C1 and (w1.bufferC1 < 2 or w1.bufferC1 < 2 or w1.bufferC1 < 2):
        insp1.block = 0
    if (insp2.C2 and w2.bufferC2 < 2) or (insp2.C3 and w3.bufferC3 < 2):
        insp2.block = 0
    
    #Finishing the Simulation
    count += 1
    if Clock >= 10000:
        break
    if count > 100000:
        break
    
print('w1 C1 buffer',w1.bufferC1)
print('w2 C1 buffer',w2.bufferC1)
print('w2 C2 buffer',w2.bufferC2)
print('w3 C1 buffer',w3.bufferC1)
print('w3 C3 buffer',w3.bufferC3)
print('w1 busy',w1.stat)
print('w2 busy',w2.stat)
print('w3 busy',w3.stat)
print('insp 1 blocked', insp1.block)
print('insp 2 blocked', insp2.block)
Events.tail()

w1 C1 buffer 0
w2 C1 buffer 0
w2 C2 buffer 2
w3 C1 buffer 0
w3 C3 buffer 2
w1 busy 1
w2 busy 0
w3 busy 0
insp 1 blocked 0
insp 2 blocked 0


,Clock,System State,FEQ,Entity,Event,Production
4336,9971.69,"[w1=1, w1_buffer_C1=0, w2=0, w2_bufferC1=0, w2...",Clock Entity Event 0 10004.942...,w1,Starting w1,"[w1_Prod=736, w2_Prod=86, w3_Prod=14]"
4337,9979.35,"[w1=0, w1_buffer_C1=0, w2=0, w2_bufferC1=0, w2...",Clock Entity Event 0 10004.942...,w1,Finish w1,"[w1_Prod=737, w2_Prod=86, w3_Prod=14]"
4338,10003.8,"[w1=0, w1_buffer_C1=0, w2=0, w2_bufferC1=0, w2...",Clock Entity Event 0 10004.942...,insp1,Finish C1,"[w1_Prod=737, w2_Prod=86, w3_Prod=14]"
4339,10003.8,"[w1=0, w1_buffer_C1=1, w2=0, w2_bufferC1=0, w2...",Clock Entity Event 0 10004.942...,insp1,Starting C1,"[w1_Prod=737, w2_Prod=86, w3_Prod=14]"
4340,10003.8,"[w1=1, w1_buffer_C1=0, w2=0, w2_bufferC1=0, w2...",Clock Entity Event 0 10004.942...,w1,Starting w1,"[w1_Prod=737, w2_Prod=86, w3_Prod=14]"


## Results of Simulation

In [9]:
print('w1 Production', w1.Production)
print('w1 % of working time = {:2.2%}'.format(w1.work_time/Clock))
print('w2 Production', w2.Production)
print('w2 % of working time = {:2.2%}'.format(w2.work_time/Clock))
print('w3 Production =', w3.Production)
print('w3 % of working time = {:2.2%}'.format(w3.work_time/Clock))
print('Overall effectiveness = {:2.2%}'.format((1/3)*((w1.work_time + w2.work_time + w3.work_time)/Clock)))
print('')
print('Inspector 1 working rate = {:2.2%}'.format(insp1.work_time/Clock))
print('Inspector 2 working rate = {:2.2%}'.format(insp2.work_time/Clock))

w1 Production 737
w1 % of working time = 39.31%
w2 Production 86
w2 % of working time = 11.31%
w3 Production = 14
w3 % of working time = 1.54%
Overall effectiveness = 17.39%

Inspector 1 working rate = 99.85%
Inspector 2 working rate = 99.48%


In [10]:
count

2170